In [0]:
import os
import time
import random
import numpy as np
import matplotlib.pyplot as plt
import pybullet_envs
import gym
import torch
import torch.nn as nn
import torch.nn.functional as F
from gym import wrappers
from torch.autograd import Variable
from collections import deque

In [0]:
class ReplayBuffer(object):
def __init__(self, max_size = 1e6):
    self.storage = []
    self.max_size = max_size
    self.ptr = 0
def add(self, transition):
if len(self.storage) == self.max_size:
    self.storage[int(self.ptr)] = transition
    self.ptr = (self.ptr + 1) % self.max_size
else:
    self.storage.append(transition)
def sample(self, batch_size):
    ind = np.random.randint(0, len(self.storage), batch_size)
    batch_states, batch_next_states, batch_actions, batch_rewards, \
    batch_dones = [], [], [], [], []
for i in ind:
state, next_state, action, reward, done = self.storage[i]
batch_states.append(np.array(state, copy = False))
batch_next_states.append(np.array(next_state, copy = False))
batch_actions.append(np.array(action, copy = False))
batch_rewards.append(np.array(reward, copy = False))
batch_dones.append(np.array(done, copy = False))
return np.array(batch_states), np.array(batch_next_states), \
np.array(batch_actions), np.array(batch_rewards).reshape(-1, 1), \
np.array(batch_dones).reshape(-1, 1)

In [0]:
class Actor(nn.Module):

def _init__(self, state_dims, action_dim, max_action):
#max action is te clip in case we gdded too much maise
super(Actor, self). init__() # activate the inheritance
self.layer_1 = nn.Linear(state_dims, 400)
self.layer_2 = nn.Linear(400, 300)
self.layer_3 = nn.Linear(300, action_dim)
self.max_action = max_action

def forward(self, x):
x = F.relu(self.layer_1(x))
x = F.relu(self.layer_2(x))
x = self.max_action * torch.tanh(self.layer_3(x))
return (x)

In [0]:
class Critic(nn.Module):

def _init_ (self, state_dims, action_dim):
#max_actton ts to clip tn case we added too much natse
super(Critic, self). init__() # activate the inheritance
# First Critic Networr
self.layer_1 = nn.Linear(state_dims + action_dim, 400)
self.layer_2 = nn.Linear(400, 300)
self.layer_3 = nn.Linear(30@, action_dim)
# Second Critic Network
self.layer_4 = nn.Linear(state_dims + action_dim, 400)
self.layer_5 = nn.Linear(400, 300)
self.layer_6 = nn.Linear(30@, action_dim)

def forward(self, x, u}: # x - state, u = actian
xu = torch.cat([x, u], 1) # 1 for vertical concatenation, @ for Hzntl
# forward propagation on first Critic
x1 = F.relu(self.layer_1(xu))
x1 = F.relu(self.layer_2(x1))
x1 = self.layer_3(x1)
# forward propagation an second Critic
x2 = F.relu(self.layer_4(xu))
x2 = F.relu(self.layer_5(x2))
x2 = self.layer_6(x2)
return x1, x2

def Ql(self, x, u): # * - state, u = action This is used far updating the @ values
xu = torch.cat([x, u], 1) # 1 for vertical concatenation, @ for Hzntl
x1 = F.relu(self.layer_1(xu))
x1 = F.relu(self.layer_2(x1))
x1 = self.layer_3(x1)
return (x1)

In [0]:

device = torch.device('cuda’ if torch.cuda.is_available() else ‘cpu’)
class T3D(object):
def _init__(self, state_dims, action_dim, max_action):
# making sure our T3230 class can work with any env
self.actor = Actor(state_dims, action_dim, max_action).to(device) # GD
self.actor_target = Actor(state_dims, action_dim, max_action).to(device) # Palyak Avg
self.actor_target.load_state_dict(self.actor.state dict)
# initializing with model weights to keen them same
self.actor_optimizer = torch.optim.Adam(self.actor.parameters())
self.critic = Critic(state_dims, action_dim).to(device) # G2
self.critic_target = critic(state_dims, action_dim).to(device) # Polyar Avg
self.critic_target.load_state_dict(self.critic.state dict)
# initializing with model weights to keen them same
self.critic_optimizer = torch.optim.Adam(self.critic.parameters())
self.max_action = max_action
def select_action(self, state):
state = torch.Tensor(state.reshape(1, -1)).to(device)
return self.actor(state).cpu().data.numpy().flatten()
# need to canvert to pumpy, remember clipping?

In [0]:
def train(self, replay_buffer, iterations, batch _size=100, discount=0.99,
tau = 0.005, policy_noise=0.2, noise_clip=0.5, policy _freq=2):
for it in range(iterations):
# Step 4 We sample fram a baten of transitions (5s, s', a, m) fram memory
batch_states, batch_next_states, batch_actions, batch_rewards, batch_dones \
= replay_buffer.sample(batch_size)
state = torch.Tensor(batch_states).to(device)
next_state = torch.Tensor(batch_next_states).to(device)
action = torch.Tensor(batch_actions).to(device)
reward = torch.Tensor(batch_rewards).to(device)
done = torch.Tensor(batch_dones) .to(device)

In [0]:
# Step 5: From the next state s’, the Acter target plays the next acttens a’
next_action = self.actor_target.forward(next_state)

In [0]:
# Step G: We add Gaussian maise to this next action a’ and we clamp if in a
# pange of values supported &y the environment

noise = torch.Tensor(batch_actions).data.normal_(0, policy_noise).to(device)
noise = noise.clamp(-noise_clip, noise clip)

next_action = (next_action + noise).clamp(-self.max_action, self.max_action)

In [0]:
# Step 7: The two Critic targets take each the couple (s’', @') as input and
# return two Q values, Qti(s', a’) and Qt2(s', a’) as outputs
target_Ql, target_Q2 = self.critic_target.forward(next_state, next_action)

In [0]:
# Step 8: We keen the minimum of these two Q-Values
target_Q = torch.min(target_Ql, target_Q2)

In [0]:
# Step 9: We get the final target of the two Critic model, which is:

# Qt = r + gamma*min(Qti, Qt2)

# target_Q = reward + (1 - done) * discount * target_Q

# @ = episode not over, 1 = episode over

# We can't run the above equation efficiently as some components are in Computational
# graphs and some are not. We need to make one minor modification

target_Q = reward + ((1 - done) * discount * target_Q).detach()

In [0]:
# Step 10: The two critic models take each the couple (s, a),
# as input and return two Q Values
current_Ql1, current_Q2 = self.critic.forward(state, action)

In [0]:
# Step 11: We compute the lass coming from the two Critic models
critic_loss = F.mse_loss(current_Ql, target_Q) + F.mse_loss(current_Q2, target_Q)

In [0]:
# Step 12: We backpropagate this Critic loss and update the parameters of the two Critic
# models with a Adam optimizer

self.critic_optimizer.zero_grad() #initializing the gradients to zero
critic_loss.backward() # computing the gradients

self.critic_optimizer.step() #performing the weight updates

In [0]:
# Sten 13; Once every two tteratians, we update our Actor model by
# performing gradtent ascent on the output of the first critic model
if it % policy_freq == Q:

# This ts DPG part

actor_loss = -(self.critic.Q1(state, self.actor(state)).mean())}

self.actor_optimizer.grad_zero()}

actor_loss.backward()

self.actor_optimizer.step()}

In [0]:
# Sten 14; still ance every tha iteractlans, we update the weights of the Actor target

# by Polyak averaging

for param, target_param in zip(self.actor.parameters(), self.actor_target.parameters()):
target_param.data.copy_(tau * param.data + (1 - tau) * target_param.data)

In [0]:
# Step 15; Still once every two iterations, we update the weights of the Critic target

fata) a eT ee ret Mee ere ee aT

for param, target_param in zip(self.critic.parameters(), self.critic_target.parameters()):
target_param.data.copy_(tau * param.data + (1 - tau) * target_param.data)

# 73D is done now!